# Development haskell notebook: decision tree CART

In [1]:
import Mllib.Types
-- import Mllib.Metrics
import Mllib.Utils.Features

import Data.List (sortOn)
import System.Random (randomR, newStdGen, randomRs)
-- import Mllib.Tree.Decision

In [2]:
-- disable warnings
:opt no-lint

In [3]:
:r

# Decision tree parameters

In [4]:
-- | Decision tree parameters for setup
data DecisionTreeParams = DecisionTreeParams
    { rGen            :: !StdGen  -- ^ Random generator
    , randomSplitter  :: !Bool    -- ^ If True, then chooses the best random split
                                  -- If False, then chooses the best split
    , maxDepth        :: !Int     -- ^ The maximum depth of the tree
    , minSamplesSplit :: !Int     -- ^ The minimum number of samples required
                                  -- to split a node
    , minSamplesLeaf  :: !Int     -- ^ The minimum number of samples required 
                                  -- to be at a leaf
    , maxFeatures     :: !Int     -- ^ The number of features to consider 
                                  -- when looking for the best split
    }
  deriving Show

In [5]:
-- | Default parameters for decision tree
treeSetup :: DecisionTreeParams
treeSetup 
    = DecisionTreeParams
        { rGen           = mkStdGen 42
        , randomSplitter = False
        , maxDepth       = maxBound
        , minSamplesSplit= 2
        , minSamplesLeaf = 1
        , maxFeatures    = -1
        }

# Computing impurity: Gini index

In [6]:
-- | Possible splits
--   Returns list of tuples with left and right groups respectively
possibleSplits
    :: [a]         -- ^ 
    -> [([a],[a])] -- ^
possibleSplits list
    = [(take i list, drop i list) | i <- [1..(length list - 1)]]

In [7]:
possibleSplits [1,2,3,4]

[([1],[2,3,4]),([1,2],[3,4]),([1,2,3],[4])]

In [8]:
-- | Possible splits with decision tree params
-- Returns list of tuples with left and right groups respectively
possibleSplitsWithParams
    :: DecisionTreeParams
    -> [a]         -- ^ given list
    -> [([a],[a])]
possibleSplitsWithParams params list = 
  let
    minLen = minSamplesLeaf params
    len = length list
  in
    [(take i list, drop i list) 
        | i <- [1..(len - 1)]
        , i >= minLen
        , (len - i) >= minLen
    ]

In [9]:
possibleSplitsWithParams treeSetup{minSamplesLeaf=1} [1,2,3,4,5]
possibleSplitsWithParams treeSetup{minSamplesLeaf=2} [1,2,3,4,5]
possibleSplitsWithParams treeSetup{minSamplesLeaf=3} [1,2,3,4,5]

[([1],[2,3,4,5]),([1,2],[3,4,5]),([1,2,3],[4,5]),([1,2,3,4],[5])]

[([1,2],[3,4,5]),([1,2,3],[4,5])]

[]

In [10]:
-- | Impurity criteria
type Criteria = [Double] -> Double  -- !working with proportions of classes in the group

In [11]:
-- | Gini's impurity
gini
    :: [Double] -- ^ Proportions of classes
    -> Double
gini props = 1 - sum (map (**2) props)

In [12]:
-- | Number of occurrences of the element in the list
countOccurrences
    :: Eq a
    => a   -- ^ element
    -> [(b,a)] -- ^ list
    -> Int
countOccurrences x = length . filter (\y -> x == snd y)

In [13]:
-- | Proportions of given group according to class names
-- Returns list of proportions
proportions
    :: [Int]      -- ^ class names
    -> [(a, Int)] -- ^ a group where element is (vector, label)
    -> [Double]
proportions classes group = 
    [fromIntegral (countOccurrences cl group) / totalSize | cl <- classes]
  where
    totalSize = fromIntegral $ length group

In [14]:
-- | Compute impurity of the split according to the criteria
-- ! Unique class names are needed
computeImpurity
    :: Criteria -- ^ impurity criteria
    -> [Int]    -- ^ class names
    -> ([(a, Int)] , [(a, Int)]) -- ^ split, tuple of lists or groups where element is (vector, label)
    -> Double
computeImpurity criteria uniqueClasses (lgroup, rgroup) = 
  let
    totalSize = fromIntegral $ length lgroup + length rgroup
    lGroupWeight = fromIntegral (length lgroup) / totalSize
    rGroupWeight = fromIntegral (length rgroup) / totalSize
  in
    ((criteria . proportions uniqueClasses) lgroup) * lGroupWeight 
        + ((criteria . proportions uniqueClasses) rgroup) * rGroupWeight

### Data and labels

In [15]:
x = [[1, 1]
    ,[2, 2]
    ,[1, 3]
    ,[3, 4]
    ,[4, 3]
    ]

In [16]:
y = [0
    ,0
    ,0
    ,1
    ,1
    ]

In [17]:
z = zip x y

In [18]:
x1 = [-1, -1, -1, -1]

In [19]:
y1 = [1, 0, 1, 0]

In [20]:
z1 = zip x1 y1

### TEST

In [21]:
(_, classes) = countFeatures y

In [22]:
map (computeImpurity gini classes) (possibleSplits z)

[0.4,0.26666666666666666,0.0,0.30000000000000004]

In [23]:
map (computeImpurity gini classes) (possibleSplits z1)

[0.3333333333333333,0.5,0.3333333333333333]

# Find best split

In [24]:
-- | Find the best split 
-- There is no sorting
-- Returns a split
findBestSplit
    :: [Int] -- ^ class names
    -> [a]   -- ^ vectors
    -> [Int] -- ^ labels
    -> (Double                    -- ^ gini's impurity
       ,([(a, Int)] , [(a, Int)]) -- ^ split or two groups where element is (vector, label)
       ) 
findBestSplit uniqueClasses x y =
  let
    z = zip x y -- packing vectors with labels
  in
--     snd $ -- taking split
    head $ -- taking minimum as (minImpurityValue, split)
    -- TODO: if there are several equal values then choose random
    sortOn fst $ -- sort by impurity value
    zip -- packing impurity value and split
      (map
        (computeImpurity gini uniqueClasses) 
        (possibleSplits z)) 
      (possibleSplits z)

### TEST

In [25]:
z

[([1,1],0),([2,2],0),([1,3],0),([3,4],1),([4,3],1)]

In [26]:
(_, uniqueClasses) = countFeatures y

In [27]:
findBestSplit uniqueClasses x y

(0.0,([([1,1],0),([2,2],0),([1,3],0)],[([3,4],1),([4,3],1)]))

# Find best split by feature

In [28]:
-- | Find the best split by feature at index
-- It sorts vectors with labels before computing impurity
-- Returns tuple of impurity and split
findBestSplitByFeature
    :: Int   -- ^ index of feature
    -> [Int] -- ^ class names
    -> [Vector R]   -- ^ vectors
    -> [Int] -- ^ labels
    -> (Double    -- ^ gini's impurity
       , Int      -- ^ index of feature by which vectors were sorted
       ,([(Vector R, Int)] , [(Vector R, Int)]) -- ^ split or two groups where element is (vector, label)
       ) 
findBestSplitByFeature index uniqueClasses x y =
  let
    xLists = map toList x -- transform vectors to lists
    -- TODO: look in HMatrix package:
    --   can we avoid transformation via `atIndex` or `!` ?
    zLists = sortOn ((!! index) . fst) (zip xLists y) -- packing transformed vecs with labels and sort
    zVecs  = map (\ (x, label) -> (fromList x, label)) zLists -- transform sorted lists to vectors
  in
    (\(imp, split) -> (imp, index, split)) $ -- add information about index
    head $ -- taking minimum as (minImpurityValue, split)
    -- TODO: if there are several equal values then choose random
    sortOn fst $ -- sort by impurity value
    zip -- packing impurity value and split
      (map
        (computeImpurity gini uniqueClasses) 
        (possibleSplits zVecs))
      (possibleSplits zVecs)

In [29]:
-- | Find the best split by feature at index according to tree params
-- It sorts vectors with labels before computing impurity
-- Returns tuple of impurity and split
findBestSplitByFeatureWithParams
    :: Int   -- ^ index of feature
    -> DecisionTreeParams -- ^ tree params
    -> [Int] -- ^ class names
    -> [Vector R]   -- ^ vectors
    -> [Int] -- ^ labels
    -> (Double    -- ^ gini's impurity
       , Int      -- ^ index of feature by which vectors were sorted
       ,([(Vector R, Int)] , [(Vector R, Int)]) -- ^ split or two groups where element is (vector, label)
       ) 
findBestSplitByFeatureWithParams index params uniqueClasses x y =
  let
    xLists = map toList x -- transform vectors to lists
    -- TODO: look in HMatrix package:
    --   can we avoid transformation via `atIndex` or `!` ?
    zLists = sortOn ((!! index) . fst) (zip xLists y) -- packing transformed vecs with labels and sort
    zVecs  = map (\ (x, label) -> (fromList x, label)) zLists -- transform sorted lists to vectors
  in
    (\(imp, split) -> (imp, index, split)) $ -- add information about index
    head $ -- taking minimum as (minImpurityValue, split)
    -- TODO: if there are several equal values then choose random
    sortOn fst $ -- sort by impurity value
    zip -- packing impurity value and split
      (map
        (computeImpurity gini uniqueClasses) 
        (possibleSplitsWithParams params zVecs))
      (possibleSplitsWithParams params zVecs)

### TEST

In [30]:
xVec = map fromList x

In [31]:
y

[0,0,0,1,1]

In [32]:
findBestSplitByFeature 0 uniqueClasses xVec y

(0.0,0,([([1.0,1.0],0),([1.0,3.0],0),([2.0,2.0],0)],[([3.0,4.0],1),([4.0,3.0],1)]))

In [33]:
findBestSplitByFeature 1 uniqueClasses xVec y

(0.0,1,([([1.0,1.0],0),([2.0,2.0],0),([1.0,3.0],0)],[([4.0,3.0],1),([3.0,4.0],1)]))

# Compute border of given tuple: (imp, index, split)

In [34]:
computeBorder
    :: (Double    -- ^ gini's impurity
       , Int      -- ^ index of feature by which vectors were sorted
       ,([(Vector R, Int)] , [(Vector R, Int)]) -- ^ split or two groups where element is (vector, label)
       ) 
    -> R -- ^ border for predicate
computeBorder (_, index, (lgroup, rgroup)) = (lastLeftValue + firstRightValue) / 2
  where
    lastLeftValue   = (!!index) $ toList $ fst $ last lgroup
    firstRightValue = (!!index) $ toList $ fst $ head rgroup

### TEST

In [35]:
indexTest1 = 0
splitTest1 = findBestSplitByFeature indexTest1 uniqueClasses xVec y
splitTest1

(0.0,0,([([1.0,1.0],0),([1.0,3.0],0),([2.0,2.0],0)],[([3.0,4.0],1),([4.0,3.0],1)]))

In [36]:
computeBorder splitTest1

2.5

In [37]:
indexTest2 = 1
splitTest2 = findBestSplitByFeature indexTest2 uniqueClasses xVec y
splitTest2

(0.0,1,([([1.0,1.0],0),([2.0,2.0],0),([1.0,3.0],0)],[([4.0,3.0],1),([3.0,4.0],1)]))

In [38]:
computeBorder splitTest2

3.0

# Make the best split

In [39]:
-- | Make best split
-- ! Sizes of vectors must be equal
-- ! It evaluates all best splits but takes first with min impurity
makeBestSplit
    :: [Int]      -- ^ class names
    -> [Vector R] -- ^ vectors
    -> [Int]      -- ^ labels
    -> (Double    -- ^ gini's impurity
       , Int      -- ^ index of feature by which vectors were sorted
       ,([(Vector R, Int)] , [(Vector R, Int)]) -- ^ split or two groups where element is (vector, label)
       ) 
makeBestSplit uniqueClasses x y = -- map ,  findBestSplitByFeature [ | i <- 0..(vecSize-1)]
  let
    vecSize = size $ head xVec
  in 
    head $ -- taking minimum as (minImpurityValue, index, split)
    -- TODO: if there are several equal values then choose random
    sortOn 
      (\(imp, index, split) -> imp) -- sort by impurity value
      [findBestSplitByFeature i uniqueClasses x y | i <- [0..(vecSize-1)]]

In [40]:
-- | Make splits for each feature of vectors
makeAllBestSplits
    :: [Int]      -- ^ class names
    -> [Vector R] -- ^ vectors
    -> [Int]      -- ^ labels
    -> [(Double   -- ^ gini's impurity
       , Int      -- ^ index of feature by which vectors were sorted
       ,([(Vector R, Int)] , [(Vector R, Int)]) -- ^ split or two groups where element is (vector, label)
       )] -- ^ list of splits for each feature
makeAllBestSplits uniqueClasses x y =
  let
    vecSize = size $ head xVec
  in 
    sortOn 
      (\(imp, index, split) -> imp) -- sort by impurity value
      [findBestSplitByFeature i uniqueClasses x y | i <- [0..(vecSize-1)]]

In [41]:
-- | Make splits for spicific features of vectors
-- according to decision tree params
makeBestSplitsForFeatures
    :: DecisionTreeParams -- ^ tree parameters
    -> [Int]      -- ^ class names
    -> [Int]      -- ^ list of feature indices
    -> [Vector R] -- ^ vectors
    -> [Int]      -- ^ labels
    -> [(Double   -- ^ gini's impurity
       , Int      -- ^ index of feature by which vectors were sorted
       ,([(Vector R, Int)] , [(Vector R, Int)]) -- ^ split or two groups where element is (vector, label)
       )] -- ^ list of splits for each feature
makeBestSplitsForFeatures params uniqueClasses indices x y  
    = sortOn 
        (\(imp, index, split) -> imp) -- sort by impurity value
        [findBestSplitByFeatureWithParams i params uniqueClasses x y | i <- indices]

### TEST

In [42]:
x
xVec
y

[[1,1],[2,2],[1,3],[3,4],[4,3]]

[[1.0,1.0],[2.0,2.0],[1.0,3.0],[3.0,4.0],[4.0,3.0]]

[0,0,0,1,1]

In [43]:
(_, uniqueClasses) = countFeatures y
uniqueClasses

[0,1]

In [44]:
makeBestSplit uniqueClasses xVec y

(0.0,0,([([1.0,1.0],0),([1.0,3.0],0),([2.0,2.0],0)],[([3.0,4.0],1),([4.0,3.0],1)]))

In [45]:
[split1, split2] = makeAllBestSplits uniqueClasses xVec y
split1
split2

(0.0,0,([([1.0,1.0],0),([1.0,3.0],0),([2.0,2.0],0)],[([3.0,4.0],1),([4.0,3.0],1)]))

(0.0,1,([([1.0,1.0],0),([2.0,2.0],0),([1.0,3.0],0)],[([4.0,3.0],1),([3.0,4.0],1)]))

# Support functions

In [46]:
-- TODO: this is needed if we want to choose from all best splits with equal impurity
-- ? finish it
getAllMinElemsFromSortedList
    :: Ord b
    => (a -> b)
    -> [a]
    -> [a]
getAllMinElemsFromSortedList getElem (x:xs) = (x:xs)

In [47]:
-- | Choose one element from the list randomly
chooseRandomElem
    :: [a]    -- ^ List
    -> StdGen -- ^ RandomGen
    -> a
chooseRandomElem xs rGen 
    | len == 1  = xs !! 0
    | len >  1  = xs !! rIndex
    | otherwise = error "ERR: chooseRandomElem: null list"
  where
    len = length xs
    (rIndex, _) = randomR (0, (len-1)) rGen

In [48]:
-- | Support function. Returns unique indices from infinite sequence.
-- !!! May get stuck in an infinite loop,
-- if required number of unique elements 
-- more than total number of unique elements.
-- TODO: make it safer
takeUniqueIndices
    :: Eq a 
    => Int -- ^ Number of indices
    -> [a] -- ^ Sequence of random indices
    -> [a] 
takeUniqueIndices n list = takeUnique n list []
  where
    takeUnique 0 xs ys = ys
    takeUnique n (x:xs) ys
        | (elem x ys) = takeUnique n xs ys
        | otherwise   = takeUnique (pred n) xs (x:ys)

In [49]:
-- | Support function. Returns unique indices from infinite sequence except one elem. 
-- !!! May get stuck in an infinite loop,
-- if required number of unique elements 
-- more than total number of unique elements.
-- TODO: make it safer
takeUniqueExceptElem
    :: Eq a 
    => Int -- ^ Number of indices
    -> [a] -- ^ Sequence of random indices
    -> a   -- ^ Except this element
    -> [a] 
takeUniqueExceptElem n list ex = takeUnique n list ex []
  where
    takeUnique 0 xs ex ys = ys
    takeUnique n (x:xs) ex ys
        | (elem x ys) || (x == ex) 
            = takeUnique n xs ex ys
        | otherwise
            = takeUnique (pred n) xs ex (x:ys)

### TEST

In [50]:
gen <- newStdGen
chooseRandomElem [1,2,3,4,5] gen

1

In [51]:
gen <- newStdGen
chooseRandomElem (makeAllBestSplits uniqueClasses xVec y) gen

(0.0,0,([([1.0,1.0],0),([1.0,3.0],0),([2.0,2.0],0)],[([3.0,4.0],1),([4.0,3.0],1)]))

In [52]:
gen <- newStdGen
takeUniqueIndices 5 (randomRs (0, 10 - 1) gen)

[5,4,8,7,0]

In [53]:
gen <- newStdGen
takeUniqueExceptElem 3 (randomRs (0, 5 - 1) gen) 0

[2,3,4]

# Decision tree

In [54]:
data DecisionTree =
    DecisionTreeNode
        { params         :: !DecisionTreeParams -- ^ Parameters for splitting  
        
        , currentDepth   :: !Int        -- ^ Current depth of the tree
        , vectorNumber   :: !Int        -- ^ Number of vectors
        , classesNumber  :: !Int        -- ^ Number of classes
        , classes        :: ![Int]      -- ^ Classes names

        , featureIndex   :: !Int         -- ^ Index of feature for the node condition
        , nodeCondition  :: !Double      -- ^ 
        
        , subForest      :: !(DecisionTree, DecisionTree) -- ^ 
        }
    | DecisionTreeLeaf
        { params         :: !DecisionTreeParams -- ^ Parameters for splitting              
        
        , currentDepth   :: !Int        -- ^ Current depth of the tree
        , vectorNumber   :: !Int        -- ^ Number of vectors
        , classesNumber  :: !Int        -- ^ Number of classes
        , classes        :: ![Int]      -- ^ Classes names
        
        , classLabel     :: !Int        -- ^ Result class
        
        , vectors        :: [Vector R] -- ^ List of vectors
        , labels         :: [Int]      -- ^ Labels of vectors
        }

In [55]:
instance Show DecisionTree where
    show (DecisionTreeLeaf params currentDepth vectorNumber classesNumber classes classLabel vectors labels) =
        "#TreeLeaf curDepth: " ++ show currentDepth ++ 
        " classLabel: " ++ show classLabel ++ 
        " vectors: " ++ show vectors ++ 
        " labels: " ++ show labels
    show (DecisionTreeNode params currentDepth vectorNumber classesNumber classes featureIndex nodeCondition (lTree, rTree)) =
        "#TreeNode curDepth: " ++ show currentDepth ++ 
        " predicate: " ++ "x" ++ show featureIndex ++ " < "++ show nodeCondition ++
              "\n" ++ concat (replicate (currentDepth+1) "  ") ++ "#Left" ++ show lTree ++
              "\n" ++ concat (replicate (currentDepth+1) "  ") ++ "#Right" ++ show rTree

In [56]:
-- | Validation of decision tree parameters
-- Params must make sense
validateParams
    :: DecisionTreeParams -- ^ Tree parameters
    -> Int                -- ^ Dimension of vectors
    -> DecisionTreeParams
validateParams params dimension
    -- TODO: add more checks for errors
    | maxFeatures params == (-1)
        = params{ maxFeatures=(truncate (sqrt (fromIntegral dimension))) }
    | maxFeatures params < 1
        = error "ERR: invalid params"
    | otherwise
        = params

In [57]:
-- | Check stop conditions of decision tree
checkStopCond
    :: DecisionTreeParams -- ^ Tree parameters
    -> Int                -- ^ Current depth
    -> [Vector R]         -- ^ List of vectors
    -> [Int]              -- ^ Labels of vectors
    -> Bool
checkStopCond params curDepth x y = 
    (maxDepth params) == curDepth
    || classesNumber == 1
    || lenX == 1
    || lenX < (minSamplesSplit params)
    || lenX < (2 * minSamplesLeaf params)
  where
    (classesNumber, classes) = countFeatures y
    lenX = length x

In [58]:
-- | Get a classification label by majority vote
mode
    :: [Int] -- ^ Names of classes
    -> [Int] -- ^ Labels of vectors
    -> Int
mode classes y = 
    fst $ last $ -- take class label of maximum
    -- TODO: solve problem with several maximums
    sortOn (snd) -- sort by quantity
    [(cl, length (filter (==cl) y)) | cl <- classes] -- take class label and make tuple with its quantity

In [59]:
-- | Generate a list of features for splits
-- according to the index of previous split
generateFeatures
    :: DecisionTreeParams -- ^ tree parameters
    -> Int        -- ^ feature index of previous split
    -> Int        -- ^ dimension of vectors
    -> [Int]      -- ^ labels
generateFeatures params prevIndex dimension 
    | prevIndex == -1 
        -- then it is the first split, we can consider all features
        = takeUniqueIndices
            (maxFeatures params)
            (randomRs (0, dimension - 1) (rGen params))
    | dimension == 1
        -- then split by the same feature index, it is 0
        = [0]
    | maxFeatures params == dimension 
        -- then take all indices except the previous
        = filter (/= prevIndex) $ [0..(dimension-1)]
    | maxFeatures params <  dimension
        -- then we can exclude previous index
        = takeUniqueExceptElem -- take elements
            (maxFeatures params) -- number of elements
            (randomRs (0, dimension - 1) (rGen params)) -- random sequence
            prevIndex -- except this element
    | otherwise
        = error "ERR: generateFeatures: maxFeatures > dimension of vector"

In [60]:
-- | Make split according to decision tree params and feature index of previous split
makeSplit
    :: DecisionTreeParams -- ^ tree parameters
    -> Int        -- ^ feature index of previous split
    -> [Int]      -- ^ class names
    -> [Vector R] -- ^ vectors
    -> [Int]      -- ^ labels
    -> (Double    -- ^ gini's impurity
       , Int      -- ^ index of feature by which vectors were sorted
       ,([(Vector R, Int)] , [(Vector R, Int)]) -- ^ split or two groups where element is (vector, label)
       ) -- ^ split
makeSplit params prevIndex uniqueClasses x y 
    | randomSplitter params 
        = chooseRandomElem 
            (makeBestSplitsForFeatures params uniqueClasses features x y) 
            (rGen params)
    | otherwise 
        = head
            (makeBestSplitsForFeatures params uniqueClasses features x y)
  where
    gen      = rGen params
    dimension = size $ head x
    features = generateFeatures params prevIndex dimension

In [61]:
-- | Main internal function from `fitDecisionTree`
-- Builds decision tree node or leaf according to stop conditions
buildDecisionTree
    :: DecisionTreeParams -- ^ Tree parameters
    -> Int                -- ^ Current depth
    -> Int                -- ^ Feature index of previous split
    -> [Vector R]         -- ^ List of vectors
    -> [Int]              -- ^ Labels of vectors
    -> DecisionTree
buildDecisionTree params curDepth prevIndex x y 
    | checkStopCond params curDepth x y = 
        DecisionTreeLeaf
            { params = params

            , currentDepth = curDepth
            , vectorNumber = length x
            , classesNumber = classesNumber
            , classes = classes

            , classLabel = mode classes y

            , vectors = x
            , labels = y
            }
    | otherwise = 
        DecisionTreeNode
            { params = params

            , currentDepth = curDepth
            , vectorNumber = length x
            , classesNumber = classesNumber
            , classes = classes

            , featureIndex = index
            , nodeCondition = computeBorder split

            , subForest = 
                (buildDecisionTree params (succ curDepth) index xLeft  yLeft
                ,buildDecisionTree params (succ curDepth) index xRight yRight)
            }
  where
    (classesNumber, classes) = countFeatures y
    split = makeSplit params prevIndex classes x y
    (_, index, (lGroup, rGroup)) = split
    (xLeft, yLeft) = unzip lGroup
    (xRight, yRight) = unzip rGroup

### Fit function

In [62]:
-- | Decision tree fit function
fitDecisionTree 
    :: DecisionTreeParams -- ^ Tree parameters
    -> [Vector R]         -- ^ List of vectors
    -> [Int]              -- ^ Labels of vectors
    -> DecisionTree
fitDecisionTree params x y 
    = buildDecisionTree (validateParams params dimension) 1{-current depth-} (-1){-feature index-} x y
  where
    dimension = size $ head x

### TEST

In [63]:
x = [[1, 1]
    ,[2, 2]
    ,[1, 3]
    ,[3, 4]
    ,[4, 3.5]
    ,[6,6]
    ,[9,9]
    ]
length x

7

In [64]:
y = [0
    ,1
    ,1
    ,2
    ,2
    ,3
    ,3
    ]
length y

7

In [65]:
x

[[1.0,1.0],[2.0,2.0],[1.0,3.0],[3.0,4.0],[4.0,3.5],[6.0,6.0],[9.0,9.0]]

In [66]:
y

[0,1,1,2,2,3,3]

In [67]:
gen <- newStdGen
fitDecisionTree treeSetup {rGen=gen, randomSplitter=False, minSamplesLeaf=1} (map vector x) y

#TreeNode curDepth: 1 predicate: x0 < 5.0
    #Left#TreeNode curDepth: 2 predicate: x1 < 3.25
      #Left#TreeNode curDepth: 3 predicate: x0 < 1.0
        #Left#TreeLeaf curDepth: 4 classLabel: 0 vectors: [[1.0,1.0]] labels: [0]
        #Right#TreeLeaf curDepth: 4 classLabel: 1 vectors: [[1.0,3.0],[2.0,2.0]] labels: [1,1]
      #Right#TreeLeaf curDepth: 3 classLabel: 2 vectors: [[4.0,3.5],[3.0,4.0]] labels: [2,2]
    #Right#TreeLeaf curDepth: 2 classLabel: 3 vectors: [[6.0,6.0],[9.0,9.0]] labels: [3,3]

In [68]:
x1 =
    [[1  , 1]
    ,[1  , 2]
    ,[3  , 1.5]
    ,[1.5, 4]
    ,[3  , 4]
    ,[4  , 3.5]
    ,[4 , 1]
    ]
length x1
y1 =
    [1
    ,1
    ,1
    ,2
    ,2
    ,2
    ,2
    ]
length y1

7

7

In [69]:
gen <- newStdGen
fitDecisionTree treeSetup {rGen=gen, randomSplitter=False} (map vector x1) y1

#TreeNode curDepth: 1 predicate: x1 < 2.75
    #Left#TreeNode curDepth: 2 predicate: x0 < 3.5
      #Left#TreeLeaf curDepth: 3 classLabel: 1 vectors: [[1.0,1.0],[1.0,2.0],[3.0,1.5]] labels: [1,1,1]
      #Right#TreeLeaf curDepth: 3 classLabel: 2 vectors: [[4.0,1.0]] labels: [2]
    #Right#TreeLeaf curDepth: 2 classLabel: 2 vectors: [[4.0,3.5],[1.5,4.0],[3.0,4.0]] labels: [2,2,2]

In [70]:
gen <- newStdGen
fitDecisionTree treeSetup {rGen=gen, randomSplitter=True} (map vector x1) y1

#TreeNode curDepth: 1 predicate: x1 < 2.75
    #Left#TreeNode curDepth: 2 predicate: x0 < 3.5
      #Left#TreeLeaf curDepth: 3 classLabel: 1 vectors: [[1.0,1.0],[1.0,2.0],[3.0,1.5]] labels: [1,1,1]
      #Right#TreeLeaf curDepth: 3 classLabel: 2 vectors: [[4.0,1.0]] labels: [2]
    #Right#TreeLeaf curDepth: 2 classLabel: 2 vectors: [[4.0,3.5],[1.5,4.0],[3.0,4.0]] labels: [2,2,2]

# Predict

In [71]:
{-# LANGUAGE NamedFieldPuns #-}

In [72]:
-- Predict class label for one vector
predictElem
    :: DecisionTree     -- ^ Fitted decision tree
    -> Vector R         -- ^ Vector
    -> Int              -- ^ Predicted label for vector
predictElem tree x = case tree of
    DecisionTreeLeaf {classLabel} 
        -> classLabel
    DecisionTreeNode {featureIndex, nodeCondition, subForest = (lTree, rTree)} 
        | xi < nodeCondition -> predictElem lTree x
        | otherwise          -> predictElem rTree x
      where
        xi = (toList x) !! featureIndex

In [73]:
-- Predict class labels for vectors
predict
    :: DecisionTree     -- ^ Fitted decision tree
    -> [Vector R]       -- ^ Vectors
    -> [Int]            -- ^ Predicted labels for vectors
predict tree x = map (predictElem tree) x

### TEST

In [74]:
x = [[0, 0]
    ,[1, 1]
    ]

y = [0, 1]

xTest = [[2, 2]]

In [75]:
tree = fitDecisionTree treeSetup (map vector x) y
tree

#TreeNode curDepth: 1 predicate: x0 < 0.5
    #Left#TreeLeaf curDepth: 2 classLabel: 0 vectors: [[0.0,0.0]] labels: [0]
    #Right#TreeLeaf curDepth: 2 classLabel: 1 vectors: [[1.0,1.0]] labels: [1]

In [76]:
predict tree (map vector xTest)

[1]

In [77]:
x =
    [[1  , 1]
    ,[1  , 2]
    ,[3  , 1.5]
    ,[1.5, 4]
    ,[3  , 4]
    ,[4  , 3.5]
    ,[4 , 1.2]
    ]
length x
y =
    [1
    ,1
    ,1
    ,2
    ,2
    ,2
    ,2
    ]
length y

7

7

In [78]:
gen <- newStdGen
tree = fitDecisionTree treeSetup {rGen=gen, randomSplitter=True} (map vector x) y
tree

#TreeNode curDepth: 1 predicate: x0 < 3.0
    #Left#TreeNode curDepth: 2 predicate: x1 < 3.0
      #Left#TreeLeaf curDepth: 3 classLabel: 1 vectors: [[1.0,1.0],[3.0,1.5],[1.0,2.0]] labels: [1,1,1]
      #Right#TreeLeaf curDepth: 3 classLabel: 2 vectors: [[1.5,4.0]] labels: [2]
    #Right#TreeLeaf curDepth: 2 classLabel: 2 vectors: [[3.0,4.0],[4.0,3.5],[4.0,1.2]] labels: [2,2,2]

In [79]:
gen <- newStdGen
tree = fitDecisionTree treeSetup {rGen=gen, randomSplitter=True, minSamplesSplit=4, minSamplesLeaf=2} (map vector x) y
tree

#TreeNode curDepth: 1 predicate: x1 < 2.75
    #Left#TreeNode curDepth: 2 predicate: x0 < 2.0
      #Left#TreeLeaf curDepth: 3 classLabel: 1 vectors: [[1.0,1.0],[1.0,2.0]] labels: [1,1]
      #Right#TreeLeaf curDepth: 3 classLabel: 2 vectors: [[3.0,1.5],[4.0,1.2]] labels: [1,2]
    #Right#TreeLeaf curDepth: 2 classLabel: 2 vectors: [[4.0,3.5],[1.5,4.0],[3.0,4.0]] labels: [2,2,2]

In [80]:
xTest = [[1,2]]
predict tree (map vector xTest)

[1]

In [81]:
xTest = [[4,2]]
predict tree (map vector xTest)

[2]

In [82]:
tree = fitDecisionTree treeSetup (map vector x) y
tree

#TreeNode curDepth: 1 predicate: x0 < 3.0
    #Left#TreeNode curDepth: 2 predicate: x1 < 3.0
      #Left#TreeLeaf curDepth: 3 classLabel: 1 vectors: [[1.0,1.0],[3.0,1.5],[1.0,2.0]] labels: [1,1,1]
      #Right#TreeLeaf curDepth: 3 classLabel: 2 vectors: [[1.5,4.0]] labels: [2]
    #Right#TreeLeaf curDepth: 2 classLabel: 2 vectors: [[3.0,4.0],[4.0,3.5],[4.0,1.2]] labels: [2,2,2]

In [88]:
x = [[1], [2], [3], [4]]
y = [0, 1, 2, 3]
tree = fitDecisionTree treeSetup (map vector x) y
tree
predict tree (map vector [[0], [1.6], [3.3], [10]])

#TreeNode curDepth: 1 predicate: x0 < 1.5
    #Left#TreeLeaf curDepth: 2 classLabel: 0 vectors: [[1.0]] labels: [0]
    #Right#TreeNode curDepth: 2 predicate: x0 < 2.5
      #Left#TreeLeaf curDepth: 3 classLabel: 1 vectors: [[2.0]] labels: [1]
      #Right#TreeNode curDepth: 3 predicate: x0 < 3.5
        #Left#TreeLeaf curDepth: 4 classLabel: 2 vectors: [[3.0]] labels: [2]
        #Right#TreeLeaf curDepth: 4 classLabel: 3 vectors: [[4.0]] labels: [3]

[0,1,2,3]